In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td

import pytorch_lightning as pl

import faiss
import tqdm
import json
import sklearn.metrics as sm

import tensorboardX as tb

# import matplotlib.pyplot as pl
# import seaborn as sns

np.random.seed(31337)

In [2]:
DATA_DIR = "data/"

In [3]:
data = pd.read_pickle(DATA_DIR + "pairs.pkl")

In [4]:
rdm = np.random.random(len(data))
train_data = data[rdm < 0.8]
val_data = data[(rdm >= 0.8) & (rdm < 0.9)]
test_data = data[rdm >= 0.9]

len(train_data), len(val_data), len(test_data)

(733123, 90877, 91993)

## Train Model

In [5]:
features = ["start", "track"]

In [6]:
class ContextualRanker(pl.LightningModule):
    def __init__(self, embedding_dim=10):
        super().__init__()
        self.embedding_dim = embedding_dim
        
        # We won't have embeddings for everything, but that's ok
        self.context = nn.Embedding(50000, embedding_dim)
        self.track = nn.Embedding(50000, embedding_dim)

    def forward(self, x):
        context = self.context(x[:, 0])
        track = self.context(x[:, 1])
        return torch.sum(context * track, dim=1)
            
    
    def step(self, batch, batch_idx, metric, prog_bar=False):
        x, y = batch
        predictions = self.forward(x)
        loss = F.mse_loss(predictions, y, reduction='mean')
        self.log(metric, loss, prog_bar=prog_bar)
        return loss

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train_loss")
    
    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "val_loss", True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
        scheduler = {
            'scheduler': lr_scheduler,
            'reduce_on_plateau': True,
            'monitor': 'val_loss'
        }
        return [optimizer], [scheduler]

In [7]:
train_dataset = td.TensorDataset(
    torch.from_numpy(train_data[features].values), 
    torch.from_numpy(train_data["time"].values)
)

val_dataset = td.TensorDataset(
    torch.from_numpy(val_data[features].values), 
    torch.from_numpy(val_data["time"].values)
)

In [8]:
net = ContextualRanker()

checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss")

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1, 
    callbacks=[
        pl.callbacks.early_stopping.EarlyStopping(monitor="val_loss", patience=5),
        pl.callbacks.LearningRateMonitor(logging_interval="step"),
        checkpoint_callback
    ])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [9]:
trainer.fit(
    net, 
    td.DataLoader(train_dataset, batch_size=2048, shuffle=True, num_workers=5),
    td.DataLoader(val_dataset, batch_size=2048, num_workers=5)
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type      | Params
--------------------------------------
0 | context | Embedding | 500 K 
1 | track   | Embedding | 500 K 
--------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.000     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [10]:
best = ContextualRanker.load_from_checkpoint(checkpoint_callback.best_model_path).to(torch.device("cuda"))

In [11]:
test_loader = td.DataLoader(torch.from_numpy(test_data[features].values), batch_size=512, shuffle=False, num_workers=5)

nn_predictions = []
for batch in tqdm.tqdm(test_loader, total=test_data.shape[0] / 512):
    nn_predictions.extend(best(batch.to(torch.device("cuda"))).cpu().detach().numpy().tolist())

 50%|█████     | 90/179.673828125 [00:00<00:00, 307.59it/s]/home/n.anokhin/tools/anaconda3/envs/nnp/lib/python3.6/site-packages/tqdm/std.py:535: TqdmWarning: clamping frac to range [0, 1]
  colour=colour)
100%|██████████| 180/179.673828125 [00:00<00:00, 335.07it/s]


In [12]:
eval_data = test_data.copy().assign(nn = nn_predictions).assign(rdm = np.random.random(len(test_data))).assign(avg = test_data["time"].mean()).head()

In [13]:
sm.mean_squared_error(eval_data["nn"], eval_data["time"])

0.007631191536556214

In [14]:
sm.mean_squared_error(eval_data["rdm"], eval_data["time"])

0.33699564357549355

In [15]:
sm.mean_squared_error(eval_data["avg"], eval_data["time"])

0.02010436

## Compute top recommendations

In [16]:
track_meta = pd.read_json(DATA_DIR + "tracks.json", lines=True)

In [17]:
context_embeddings = dict(best.named_parameters())["context.weight"].data.cpu().numpy()
track_embeddings = dict(best.named_parameters())["track.weight"].data.cpu().numpy()

In [18]:
index = faiss.IndexFlatIP(best.embedding_dim)
index.add(track_embeddings)
print(index.ntotal)

50000


In [19]:
track_meta.head()

artist                            title  track
0   Jack Johnson                         The Cove      0
1  Billy Preston             Nothing from Nothing      1
2  Paco De Lucia                  Entre Dos Aguas      2
3     Josh Rouse            Under Cold Blue Stars      3
4   The Dead 60s  Riot Radio (Soundtrack Version)      4

In [20]:
with open(DATA_DIR + "tracks_recs.json", "w") as rf:
    for _, track in tqdm.tqdm(track_meta.iterrows()):
        embedding = context_embeddings[track["track"]]
        dists, neigbours = index.search(embedding[np.newaxis, :], 10)
        
        recommendation = dict(track)
        recommendation["recommendations"] = neigbours.tolist()
        
        rf.write(json.dumps(recommendation) + "\n")

50000it [01:12, 689.12it/s]
